In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
from dotenv import load_dotenv

In [3]:
# Load Environment Variables
load_dotenv()

True

In [4]:
# Create an engine to connect to the database

from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.asyncio import (
    AsyncSession,
    create_async_engine,
)
db_type = "sqlite"
aengine = create_async_engine(
    url="sqlite+aiosqlite:///demo_databases/tn_covid_cases_11_may.sqlite"
)

async_session = sessionmaker(
    bind=aengine,
    class_=AsyncSession,
    expire_on_commit=False
)

In [5]:
# Parameters
metric_db_id = "test"
llm = "gpt-4o"
guardrails_llm = "gpt-4o"
sys_message = "Government and health officials in Tamil Nadu, India will ask you questions. You need to help them manage COVID cases and the availablity of beds in health facilities."
db_description = "- bed_vacancies_clinics_11_may: Each row identifies a district and the beds earmarked, occupied and available for COVID cases in the district clinics.\
- bed_vacancies_health_centers_and_district_hospitals_11_may: Each row identifies a district and the beds earmarked, occupied and available, with and without oxygen supply, and with and without ICU support, for COVID cases in the disctrict health centers and hospitals.\
- covid_cases_11_may: Each row identifies a district and the number of people who received treatment, were discharged and died due to COVID.\
"
db_column_description = ""
num_common_values = 10
indicator_vars="district_name" # This should be a comma delimited string in multiple vars
context_length = 5 # <-- how many previous queries to consider for the context

# Simulate a conversation with follow-up questions
queries = [
    # 1
    {
    "query_text": "How many beds are there in chennai??",
    "query_metadata": {}},

    # 2
    {"query_text": "Order districts by bed availability in ICUs",
                            "query_metadata": {}},

    # 3
    {"query_text": "Which district has the least?",
      "query_metadata": {}},

    # 4
    {"query_text": "Which district has the most?",
      "query_metadata": {}},

    # 5
    {"query_text": "Which has the most cases?",
      "query_metadata": {}}
]

In [6]:
# Ask  a question
from askametric.query_processor.query_processor import LLMQueryProcessor

async with async_session() as session:
    qp = LLMQueryProcessor(
        asession=session,
        metric_db_id=metric_db_id,
        db_type=db_type,
        llm=llm,
        guardrails_llm=guardrails_llm,
        sys_message=sys_message,
        db_description=db_description,
        column_description="",
        num_common_values=num_common_values,
        indicator_vars=indicator_vars,
        context_length=context_length
    )
    for query in queries:
        await qp.process_query(query)
        print(f"Q: {query['query_text']}\n")
        print(f"A: {qp.user_query_response.text_response}\n\n")


In Chennai, there are a total of 20,334 beds available. This number is derived from two sources: 7,179 beds from clinics and 13,155 beds from health centers and district hospitals. The total is the sum of these two figures.
